# Pandas VS Polars

## Kütüphaneler

Yüklenmesi Gereken Kütüphaneler:
- Pandas kütüphanesi yükle değilse `!pip install pandas` diyerek yükeleyebilirsiniz.
- Numpy kütüphanesi yükle değilse `!pip install numpy` diyerek yükeleyebilirsiniz.
- Polars kütüphanesi yüklü değilse `!pip install polars` diyerek yükleyebilirsiniz.

Not: Eğer M1 veya üstü bir çipe sahip olan Mac cihazlarında polars kütüphanesini yüklemek isterseniz `pip install polars-lts-cpu` diyerek yükleyebilirsiniz.

In [1]:
import pandas as pd
import numpy as np
import polars as pl

## Örnek Veri Seti Oluşturma

In [2]:
# Yüksek boyutta bir veri oluşturalım
num_rows = 1000000
num_cols = 50

# Rastgele sayıların tekrar üretilebilmesi için seed değeri
np.random.seed(42)

# Rastgele sayılar içeren bir veriye dönüştürelim
data = np.random.randn(num_rows, num_cols)

# Kolon isimlerini oluşturalım
columns = [f"col_{i}" for i in range(num_cols)]

# Veri setlerini oluşturalım
df_pandas = pd.DataFrame(data, columns=columns)
df_polars = pl.from_numpy(data, schema=columns)

## Karşılaştırmalar

### 1) Veri Yazma ve Okuma Karşılaştırması

Not: parquet yazabilmeniz için pyarrow ve fastparquet kütüphanelerini yüklemeniz gerekmektedir. Bu şekilde yükleyebilirsiniz `!pip install pyarrow` `!pip install fastparquet`

In [3]:
# Timeit modulünün kullanılabilmesi için işlemleri fonksiyonlaştırıyoruz

# Pandas okuma fonksiyonu
def pandas_read():

    df_pandas = pd.read_parquet("data.parquet")

    return df_pandas

# Polars okuma fonksiyonu
def polars_read():

    df_polars = pl.read_parquet("data.parquet")

    return df_polars

# Pandas yazma fonksiyonu
def pandas_write(df_pandas):

    df_pandas.to_parquet("data.parquet", index=False)

# Polars yazma fonksiyonu
def polars_write(df_polars):

    df_polars.write_parquet("data.parquet")

In [4]:
%%timeit
pandas_write(df_pandas)

1.18 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
polars_write(df_polars)

773 ms ± 59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
df_pandas = pandas_read()

286 ms ± 89.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
df_polars = polars_read()

142 ms ± 4.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Veriye head atarak baktığımızda iki veri setinin de aynı olduğunu görebiliyoruz, yazma ve okuma açısından bir sorun yaşamamışlar

In [8]:
df_pandas.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,-0.469474,0.542560,...,0.738467,0.171368,-0.115648,-0.301104,-1.478522,-0.719844,-0.460639,1.057122,0.343618,-1.763040
1,0.324084,-0.385082,-0.676922,0.611676,1.031000,0.931280,-0.839218,-0.309212,0.331263,0.975545,...,0.097078,0.968645,-0.702053,-0.327662,-0.392108,-1.463515,0.296120,0.261055,0.005113,-0.234587
2,-1.415371,-0.420645,-0.342715,-0.802277,-0.161286,0.404051,1.886186,0.174578,0.257550,-0.074446,...,0.227460,1.307143,-1.607483,0.184634,0.259883,0.781823,-1.236951,-1.320457,0.521942,0.296985
3,0.250493,0.346448,-0.680025,0.232254,0.293072,-0.714351,1.865775,0.473833,-1.191303,0.656554,...,-0.446515,0.856399,0.214094,-1.245739,0.173181,0.385317,-0.883857,0.153725,0.058209,-1.142970
4,0.357787,0.560785,1.083051,1.053802,-1.377669,-0.937825,0.515035,0.513786,0.515048,3.852731,...,-0.792521,-0.114736,0.504987,0.865755,-1.200296,-0.334501,-0.474945,-0.653329,1.765454,0.404982


In [9]:
df_polars.head()

col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.496714,-0.138264,0.647689,1.52303,-0.234153,-0.234137,1.579213,0.767435,-0.469474,0.54256,-0.463418,-0.46573,0.241962,-1.91328,-1.724918,-0.562288,-1.012831,0.314247,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.95967,-1.328186,0.196861,0.738467,0.171368,-0.115648,-0.301104,-1.478522,-0.719844,-0.460639,1.057122,0.343618,-1.76304
0.324084,-0.385082,-0.676922,0.611676,1.031,0.93128,-0.839218,-0.309212,0.331263,0.975545,-0.479174,-0.185659,-1.106335,-1.196207,0.812526,1.35624,-0.07201,1.003533,0.361636,-0.64512,0.361396,1.538037,-0.035826,1.564644,-2.619745,0.821903,0.087047,-0.299007,0.091761,-1.987569,-0.219672,0.357113,1.477894,-0.51827,-0.808494,-0.501757,0.915402,0.328751,-0.52976,0.513267,0.097078,0.968645,-0.702053,-0.327662,-0.392108,-1.463515,0.29612,0.261055,0.005113,-0.234587
-1.415371,-0.420645,-0.342715,-0.802277,-0.161286,0.404051,1.886186,0.174578,0.25755,-0.074446,-1.918771,-0.026514,0.06023,2.463242,-0.192361,0.301547,-0.034712,-1.168678,1.142823,0.751933,0.791032,-0.909387,1.402794,-1.401851,0.586857,2.190456,-0.990536,-0.566298,0.099651,-0.503476,-1.550663,0.068563,-1.062304,0.473592,-0.919424,1.549934,-0.783253,-0.322062,0.813517,-1.230864,0.22746,1.307143,-1.607483,0.184634,0.259883,0.781823,-1.236951,-1.320457,0.521942,0.296985
0.250493,0.346448,-0.680025,0.232254,0.293072,-0.714351,1.865775,0.473833,-1.191303,0.656554,-0.974682,0.787085,1.158596,-0.820682,0.963376,0.412781,0.82206,1.896793,-0.245388,-0.753736,-0.889514,-0.81581,-0.077102,0.341152,0.276691,0.827183,0.013002,1.453534,-0.264657,2.720169,0.625667,-0.857158,-1.070892,0.482472,-0.223463,0.714,0.473238,-0.072829,-0.846794,-1.514847,-0.446515,0.856399,0.214094,-1.245739,0.173181,0.385317,-0.883857,0.153725,0.058209,-1.14297
0.357787,0.560785,1.083051,1.053802,-1.377669,-0.937825,0.515035,0.513786,0.515048,3.852731,0.570891,1.135566,0.954002,0.651391,-0.315269,0.758969,-0.772825,-0.236819,-0.485364,0.081874,2.314659,-1.867265,0.68626,-1.612716,-0.471932,1.088951,0.06428,-1.077745,-0.715304,0.679598,-0.730367,0.216459,0.045572,-0.6516,2.143944,0.633919,-2.025143,0.186454,-0.661786,0.852433,-0.792521,-0.114736,0.504987,0.865755,-1.200296,-0.334501,-0.474945,-0.653329,1.765454,0.404982


### 2) Kolon Operasyonları Karşılaştırması

#### Kolon Bazında Toplama, Çıkarma, Çarpma, Bölme

In [10]:
# Timeit modulünün kullanılabilmesi için işlemleri fonksiyonlaştırıyoruz

# Pandas kütüphanesi ile toplama, çıkarma, çarpma ve bölme işlemleri
def column_operation_pandas(df_pandas):
    df_pandas["col_toplama"] = df_pandas["col_0"] + df_pandas["col_1"]
    df_pandas["col_cikarma"] = df_pandas["col_0"] - 2
    df_pandas["col_carpma"] = df_pandas["col_0"] * df_pandas["col_1"]
    df_pandas["col_bolme"] = df_pandas["col_0"] / 3

# Polars kütüphanesi ile toplama, çıkarma, çarpma ve bölme işlemleri
def column_operation_polars(df_polars):
    df_polars = df_polars.with_columns((pl.col("col_0") + pl.col("col_1")).alias("col_toplama"))
    df_polars = df_polars.with_columns((pl.col("col_0") - 2).alias("col_cikarma"))
    df_polars = df_polars.with_columns((pl.col("col_0") * pl.col("col_1")).alias("col_carpma"))
    df_polars = df_polars.with_columns((pl.col("col_0") / 3).alias("col_bolme"))

In [11]:
%%timeit
column_operation_pandas(df_pandas)

26.5 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
column_operation_polars(df_polars)

1.94 ms ± 64.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Kolon Ekleme ve Düşürme

In [13]:
# Rastgele sayıların tekrar üretilebilmesi için seed değeri
np.random.seed(42)

# Kolon eklemek için ve sonraki aşamada string'den ve float'dan integer dönüşümü için rastgele veri listelerinin oluşturulması
values = np.random.randint(low=1,high=10,size=num_rows)
casting_str_to_int = values.astype(str)
casting_float_to_int = values.astype(float)

In [14]:
# Timeit modulünün kullanılabilmesi için işlemleri fonksiyonlaştırıyoruz

# Pandas kütüphanesinde hazır veri veya sabit değer ile kolon oluşturma
def column_add_pandas(df_pandas, data1, data2, return_status=False):

    df_pandas["str_to_int"] = data1
    df_pandas["float_to_int"] = data2
    df_pandas["col_value_0"] = 0

    if return_status:

        return df_pandas

# Polars kütüphanesinde hazır veri veya sabit değer ile kolon oluşturma
def column_add_polars(df_polars, data1, data2, return_status=False):

    df_polars = df_polars.with_columns(pl.lit(data1).alias("str_to_int"))
    df_polars = df_polars.with_columns(pl.lit(data2).alias("float_to_int"))
    df_polars = df_polars.with_columns(pl.lit(0).alias("col_value_0"))

    if return_status:

        return df_polars

# Pandas kütüphanesinde kolon düşürme
def column_drop_pandas(df_pandas):

    df_pandas.drop(["col_49","col_48","col_47"], axis=1)

# Polars kütüphanesinde kolon düşürme
def column_drop_polars(df_polars):

    df_polars.drop(["col_49","col_48","col_47"])

In [15]:
%%timeit
column_add_pandas(df_pandas, casting_str_to_int, casting_float_to_int)

56.2 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
column_add_polars(df_polars, casting_str_to_int, casting_float_to_int)

303 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
column_drop_pandas(df_pandas)

114 ms ± 4.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
column_drop_polars(df_polars)

16.7 μs ± 314 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### Kolon Bazında Casting

In [19]:
# Casting işlemleri için iki veri setine de integer olması gereken kolonlar ekliyoruz.
df_pandas = column_add_pandas(df_pandas, casting_str_to_int, casting_float_to_int, return_status=True)
df_polars = column_add_polars(df_polars, casting_str_to_int, casting_float_to_int, return_status=True)

In [20]:
# Timeit modulünün kullanılabilmesi için işlemleri fonksiyonlaştırıyoruz

# Pandas kütüphanesinde casting işlemlerinin yapılması
def casting_pandas(df_pandas):

    df_pandas["str_to_int"] = df_pandas["str_to_int"].astype(int)
    df_pandas["float_to_int"] = df_pandas["float_to_int"].astype(int)

# Polars kütüphanesinde casting işlemlerinin yapılması
def casting_polars(df_polars):

    df_polars = df_polars.with_columns(pl.col("str_to_int").cast(pl.Int64))
    df_polars = df_polars.with_columns(pl.col("float_to_int").cast(pl.Int64))

In [21]:
%%timeit
casting_pandas(df_pandas)

1.22 ms ± 24.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
%%timeit
casting_polars(df_polars)

7.58 ms ± 95 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 3) Gruplama İşemlerinin Karşılaştırılması

In [23]:
# Rastgele sayıların tekrar üretilebilmesi için seed değeri
np.random.seed(42)

# Group kolonu için rastgele grup sayılarının oluşturulması
groups = np.random.randint(low=1,high=10,size=num_rows)

# Group By işlemlerini gerçekleştirebilmek için iki veri setine de grup kolonu ekleyeceğiz.
df_pandas["groups"] = groups
df_polars = df_polars.with_columns(pl.lit(groups).alias("groups"))

In [24]:
# Timeit modulünün kullanılabilmesi için işlemleri fonksiyonlaştırıyoruz

# Pandas kütüphanesinde gruplama işlemlerinin yapılması
def groupby_pandas(df_pandas):

    df_pandas.groupby("groups").agg(mean_col_0=('col_0','mean'),
                                    std_col_1=('col_1','std'),
                                    sum_col_2=('col_2','sum'))

# Polars kütüphanesinde gruplama işlemlerinin yapılması
def groupby_polars(df_polars):

    df_polars.group_by("groups").agg(pl.col('col_0').mean().alias("mean_col_0"),
                                     pl.col('col_1').std().alias("std_col_1"),
                                     pl.sum('col_2').alias("sum_col_2"))

In [25]:
%%timeit
groupby_pandas(df_pandas)

57.7 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit
groupby_polars(df_polars)

7.53 ms ± 174 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 4) Birleştirme (Merge, Concatenate) İşlemleri

Bazı işlemler gerçekleştirdiğimiz için ilk dataframeleri tekrar okuyoruz

In [27]:
# Veri setlerini okuyalım
df_pandas = pd.read_parquet("data.parquet")
df_polars = pl.read_parquet("data.parquet")

Birleştirme işlemlerinde kullanılacak dataframe oluşturuyoruz.

In [28]:
# Yüksek boyutta bir veri oluşturalım
num_rows = 1000000
num_cols = 50

# Rastgele sayıların tekrar üretilebilmesi için seed değeri
np.random.seed(0)
# Rastgele sayılar içeren bir veriye dönüştürelim
data = np.random.randn(num_rows, num_cols)

# Kolon isimlerini oluşturalım
columns = [f"col_{i}" for i in range(num_cols)]

# Veri setlerini oluşturalım
df_pandas_merging = pd.DataFrame(data, columns=columns)
df_polars_merging = pl.from_numpy(data, schema=columns)

Birleştirmede kullanılacak index kolonlarını oluşturalım

In [29]:
# Veri setlerine birleştirme işlemlerinde kullanılacak isim numaralarını belirtecek verinin oluşturulması
index_numbers = np.arange(start=0, stop=num_rows, step=1)

# Index kolonunun tüm dataframe'lere eklenmesi
df_pandas["index"] = index_numbers
df_pandas_merging["index"] = index_numbers

df_polars = df_polars.with_columns(pl.lit(index_numbers).alias("index"))
df_polars_merging = df_polars_merging.with_columns(pl.lit(index_numbers).alias("index"))

#### Merge İşlemi

In [30]:
# Timeit modulünün kullanılabilmesi için işlemleri fonksiyonlaştırıyoruz

# Pandas kütüphanesinde birleştirme işlemlerinin yapılması
def merge_pandas(df_pandas, df_pandas_merging):

    df_pandas.merge(df_pandas_merging, how="left", on="index")
    df_pandas.merge(df_pandas_merging, how="right", on="index")
    df_pandas.merge(df_pandas_merging, how="inner", on="index")

# Polars kütüphanesinde birleştirme işlemlerinin yapılması
def merge_polars(df_polars, df_polars_merging):

    df_polars.join(df_polars_merging, how="left", on="index")
    df_polars.join(df_polars_merging, how="right", on="index")
    df_polars.join(df_polars_merging, how="inner", on="index")

In [31]:
%%timeit
merge_pandas(df_pandas, df_pandas_merging)

1.59 s ± 68.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%%timeit
merge_polars(df_polars, df_polars_merging)

262 ms ± 40.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Concatenate İşlemi

In [33]:
# Timeit modulünün kullanılabilmesi için işlemleri fonksiyonlaştırıyoruz

# Pandas kütüphanesinde birleştirme işlemlerinin yapılması
def concat_pandas(df_pandas, df_pandas_merging):

    _ = pd.concat([df_pandas,df_pandas_merging], axis=0)

# Polars kütüphanesinde birleştirme işlemlerinin yapılması
def concat_polars(df_polars, df_polars_merging):

    _ = pl.concat([df_polars,df_polars_merging])

In [34]:
%%timeit
concat_pandas(df_pandas, df_pandas_merging)

392 ms ± 4.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%timeit
concat_polars(df_polars, df_polars_merging)

169 μs ± 6.21 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Sonuçlar

### Genel Yorum
- Genel olarak çoğu işlemde Polars kütüphanesi Pandas kütüphanesinden çok daha hızlı

#### 1) Veri Okuma ve Yazma
- Veri Okuma: Pandas: 1.2 s, Polars: 769 ms, **Polars %56 Daha Hızlı**
- Veri Yazma: Pandas: 227 ms, Polars: 148 ms, **Polars %53 Daha Hızlı**

#### 2) Kolon Operasyonları
- Toplama, Çıkarma, Çarpma, Bölme: Pandas: 25.8ms, **Polars: 1.99ms, Polars %1196 Daha Hızlı**
- Kolon Ekleme: Pandas: 57.2 ms, Polars: 315 ms, **Pandas %451 Daha Hızlı**
- Kolon Düşürme: Pandas: 109 ms, Polars: 16.3 μs, **Polars %668611 Daha Hızlı**
- Kolon Casting: Pandas: 1.24 ms, Polars: 7.57 ms, **Pandas %510 Daha Hızlı**

#### 3) Gruplama İşlemleri
- Groupby: Pandas 56.4 ms, Polars 7.5ms, **Polars %652 Daha Hızlı**

#### 4) Birleştirme İşlemleri
- Merging: Pandas: 2.06 s, Polars: 158 ms, **Polars %1204 Daha Hızlı**
- Concatenate: Pandas: 394 ms, Polars: 162 μs, **Polars %24221 Daha Hızlı**


### Kaydedilen DataFrame'in boyutundan dolayı silinmesi (GitHub dosya sınırından kaynaklı)

In [36]:
import os
os.remove("data.parquet")